# Iceland Snow and Ice Monitoring

This notebook implements a workflow for monitoring snow and ice in Iceland using Sentinel-2 data via the EOPF Zarr format.

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
from shapely.geometry import box
import shapely.geometry
import pystac_client
from pystac import Item
import xarray as xr
import os
import requests
from datetime import datetime
from pyproj import Transformer
import dask
import warnings
from concurrent.futures import ThreadPoolExecutor, as_completed

## Seeds
Load the glacier seeds (points) and define the Area of Interest.

In [ ]:
# Load seeds
seeds_gdf = gpd.read_file("data/Iceland_Seeds.geojson")

# Reproject to WGS84 for search
seeds_gdf = seeds_gdf.to_crs("EPSG:4326")

# Get bounding box in WGS84
total_bounds = seeds_gdf.total_bounds
bbox_4326 = list(total_bounds) # [minx, miny, maxx, maxy]

# Define AOI for UTM transformation
spatial_extent = {
    "west": bbox_4326[0],
    "south": bbox_4326[1],
    "east": bbox_4326[2],
    "north": bbox_4326[3],
}

print(f"Bbox (EPSG:4326): {bbox_4326}")

# Convert AOI to UTM 27N (EPSG:32627) - Common for Iceland
# The example used EPSG:32631 for Belgium. For Iceland, we use 32627.
target_crs = "EPSG:32627"
transformer = Transformer.from_crs("EPSG:4326", target_crs, always_xy=True)

west_utm, south_utm = transformer.transform(
    spatial_extent["west"], spatial_extent["south"]
)
east_utm, north_utm = transformer.transform(
    spatial_extent["east"], spatial_extent["north"]
)

# Spatial slice parameters (Note: y is typically north-to-south in these grids, so slice order matters)
# We will verify the order after inspection, but typically it is slice(max_y, min_y) or slice(min_y, max_y) depending on the index.
# The example used slice(north_utm, south_utm) for y, implying descending coordinates.
x_slice = slice(west_utm, east_utm)
y_slice = slice(north_utm, south_utm)

print(f"UTM Bounds ({target_crs}): West={west_utm}, South={south_utm}, East={east_utm}, North={north_utm}")

## STAC Search and Data Loading

This algorithm simulates a file-based workflow by processing full Sentinel-2 scenes (Sentinel-2 L2A) retrieved from the EOPF Zarr store. 
It avoids tile-based optimization and instead loads the full scene extent to compute NDSI and classify snow.

In [ ]:
# === Configuration ===
SINGLE_SCENE_MODE = True  # Toggle: True = only first scene, False = all scenes

# STAC Search
catalog = pystac_client.Client.open("https://stac.core.eopf.eodc.eu")

# Search for Sentinel-2 L2A items
time_range_str = "2025-09-01/2025-09-30"

print(f"Searching STAC for {time_range_str} over AOI...")
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox_4326,
    datetime=time_range_str,
)

items = list(search.items())
print(f"Found {len(items)} items.")

# Filter: only non-deprecated items with 'product' asset
valid_items = [
    item for item in items 
    if not item.properties.get("deprecated", False) and "product" in item.assets
]
print(f"Valid items with product asset: {len(valid_items)}")

# Get hrefs for file-based processing
hrefs = [item.assets["product"].href for item in valid_items]

# Apply single scene toggle
if SINGLE_SCENE_MODE and hrefs:
    hrefs = [hrefs[0]]
    print(f"\n[SINGLE_SCENE_MODE] Using only first scene: {os.path.basename(hrefs[0])}")
elif hrefs:
    print(f"\nTotal scenes to process: {len(hrefs)}")

## File-Based Scene Processing

Load and process each Sentinel-2 scene individually. This simulates a file-based workflow where each scene is processed one at a time.

In [ ]:
def load_scene(href, x_slice, y_slice):
    """
    Load a single Sentinel-2 scene from EOPF Zarr store.
    Returns bands B03 (Green), B11 (SWIR), and SCL mask aligned to 10m grid.
    """
    scene_name = os.path.basename(href.rstrip("/"))
    print(f"Loading scene: {scene_name}")
    
    # Load Green Band (B03) - 10m resolution
    ds_b03 = xr.open_zarr(href, group="/measurements/reflectance/r10m")[["b03"]]
    ds_b03 = ds_b03.sel(x=x_slice, y=y_slice)
    
    # Load SWIR Band (B11) - 20m resolution
    ds_b11 = xr.open_zarr(href, group="/measurements/reflectance/r20m")[["b11"]]
    ds_b11 = ds_b11.sel(x=x_slice, y=y_slice)
    
    # Load SCL (Scene Classification) - 20m resolution
    ds_scl = xr.open_zarr(href, group="/conditions/mask/l2a_classification/r20m")[["scl"]]
    ds_scl = ds_scl.sel(x=x_slice, y=y_slice)
    
    # Resample 20m bands to 10m grid
    ds_b11_interp = ds_b11.interp_like(ds_b03, method="nearest")
    ds_scl_interp = ds_scl.interp_like(ds_b03, method="nearest")
    
    # Merge into single dataset
    scene_data = xr.merge([ds_b03, ds_b11_interp, ds_scl_interp])
    
    # Extract datetime from filename
    parts = scene_name.split("_")
    date_str = next((p for p in parts if p.startswith("20") and "T" in p), None)
    if date_str:
        scene_data.attrs["datetime"] = datetime.strptime(date_str.split(".")[0], "%Y%m%dT%H%M%S")
    
    scene_data.attrs["scene_name"] = scene_name
    return scene_data


def compute_ndsi(scene_data):
    """
    Compute NDSI (Normalized Difference Snow Index) and snow mask.
    NDSI = (Green - SWIR) / (Green + SWIR)
    Snow threshold: NDSI > 0.42
    """
    green = scene_data["b03"]
    swir = scene_data["b11"]
    scl = scene_data["scl"]
    
    # Calculate NDSI
    denom = green + swir
    ndsi = (green - swir) / denom.where(denom != 0)
    
    # Cloud mask: exclude Cloud Shadows (3), Cloud Medium (8), Cloud High (9)
    valid_mask = ~scl.isin([3, 8, 9])
    
    # Snow classification (NDSI > 0.42 and valid pixel)
    snow_mask = (ndsi > 0.42) & valid_mask
    
    return xr.Dataset({
        "ndsi": ndsi,
        "snow_mask": snow_mask,
        "valid_mask": valid_mask,
    }, attrs=scene_data.attrs)


def process_scenes(hrefs, x_slice, y_slice):
    """
    Process all scenes in file-based workflow.
    Returns list of results (one per scene).
    """
    results = []
    
    for i, href in enumerate(hrefs):
        print(f"\n[{i+1}/{len(hrefs)}] Processing...")
        try:
            # Load scene
            scene_data = load_scene(href, x_slice, y_slice)
            
            # Compute NDSI and snow mask
            result = compute_ndsi(scene_data)
            
            # Trigger computation
            result = result.compute()
            
            # Statistics
            snow_pixels = result["snow_mask"].sum().item()
            valid_pixels = result["valid_mask"].sum().item()
            snow_percent = (snow_pixels / valid_pixels * 100) if valid_pixels > 0 else 0
            
            print(f"  → Snow pixels: {snow_pixels:,} ({snow_percent:.1f}% of valid area)")
            
            results.append(result)
            
        except Exception as e:
            print(f"  → Error: {e}")
            continue
    
    return results


# Process scenes
if hrefs:
    print(f"Starting file-based processing of {len(hrefs)} scene(s)...\n")
    results = process_scenes(hrefs, x_slice, y_slice)
    print(f"\n✓ Successfully processed {len(results)} scene(s)")
else:
    print("No scenes to process.")

## Visualization: Stacked Scene Composite

Stack all processed scenes and compute median NDSI and aggregated snow mask for robust visualization.

In [ ]:
# ======================================================================
# Stack Results and Compute Composite
# ======================================================================
# Combines all processed scenes into a single composite using median/mean

NDSI_THRESHOLD = 0.42  # Standard snow threshold

def create_stacked_composite(results):
    """
    Stack all scene results and compute composite metrics.
    - Median NDSI (robust against outliers/clouds)
    - Aggregated snow mask (snow if detected in majority of valid scenes)
    """
    if not results:
        return None
    
    scene_count = len(results)
    print(f"Stacking {scene_count} scene(s)...")
    
    if scene_count == 1:
        # Single scene: use directly
        ndsi_median = results[0]["ndsi"]
        snow_mask = results[0]["snow_mask"]
        valid_mask = results[0]["valid_mask"]
    else:
        # Multiple scenes: stack along new dimension and compute median
        ndsi_stack = xr.concat([r["ndsi"] for r in results], dim="scene")
        valid_stack = xr.concat([r["valid_mask"] for r in results], dim="scene")
        
        # Median NDSI (ignoring NaN)
        ndsi_median = ndsi_stack.median(dim="scene", skipna=True)
        
        # Valid mask: pixel valid in at least one scene
        valid_mask = valid_stack.any(dim="scene")
        
        # Snow mask from median NDSI
        snow_mask = (ndsi_median > NDSI_THRESHOLD) & valid_mask
    
    # Calculate statistics
    total_pixels = valid_mask.size
    valid_pixels = valid_mask.sum().item()
    snow_pixels = snow_mask.sum().item()
    snow_percentage = (snow_pixels / valid_pixels * 100) if valid_pixels > 0 else 0
    
    print(f"  → Total pixels: {total_pixels:,}")
    print(f"  → Valid pixels: {valid_pixels:,}")
    print(f"  → Snow pixels: {snow_pixels:,} ({snow_percentage:.1f}%)")
    
    return {
        "ndsi_median": ndsi_median,
        "snow_mask": snow_mask,
        "valid_mask": valid_mask,
        "scene_count": scene_count,
        "snow_percentage": snow_percentage
    }


# Create composite from results
if 'results' in dir() and results:
    composite = create_stacked_composite(results)
else:
    composite = None
    print("No results available. Run processing cell first.")

In [ ]:
# ======================================================================
# Visualization: NDSI and Snow/Ice Classification Results
# ======================================================================
# Creates a 2-panel diagnostic plot:
# 1. Median NDSI map (spectral snow index)
# 2. Binary snow/ice mask (pixels where NDSI >= threshold)

if composite:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # ----- Plot 1: Median NDSI -----
    ndsi_data = composite['ndsi_median'].values
    im1 = axes[0].imshow(ndsi_data, cmap='RdYlGn', vmin=-0.5, vmax=1.0)
    axes[0].set_title(f'Median NDSI\n(from {composite["scene_count"]} scene(s))', 
                      fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Pixel X')
    axes[0].set_ylabel('Pixel Y')
    plt.colorbar(im1, ax=axes[0], label='NDSI', shrink=0.8)
    
    # ----- Plot 2: Snow/Ice Mask -----
    snow_data = composite['snow_mask'].values.astype(float)
    im2 = axes[1].imshow(snow_data, cmap='Blues', vmin=0, vmax=1)
    axes[1].set_title(f'Snow/Ice Mask\n(NDSI >= {NDSI_THRESHOLD})', 
                      fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Pixel X')
    axes[1].set_ylabel('Pixel Y')
    
    # Add percentage annotation
    axes[1].text(0.5, 0.95, f'{composite["snow_percentage"]:.1f}% Snow Coverage', 
                 transform=axes[1].transAxes, ha='center', va='top', fontsize=11, 
                 bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Visualization complete")
    print(f"  Scenes processed: {composite['scene_count']}")
    print(f"  Snow coverage: {composite['snow_percentage']:.1f}%")
else:
    print("No composite available. Run stacking cell first.")

## RGB Visualization (Separate Processing)

This section loads RGB bands independently from the NDSI processing to avoid data interference.

In [ ]:
# ======================================================================
# RGB Processing - Separate from NDSI
# ======================================================================
# Loads RGB bands at 20m resolution to reduce memory/container load

def load_rgb_scene(href, x_slice, y_slice):
    """
    Load RGB bands (B02, B03, B04) at 20m resolution for a single scene.
    Uses 20m bands to reduce data volume and container load.
    """
    scene_name = os.path.basename(href.rstrip("/"))
    
    # Load 20m RGB bands (lower resolution = less data)
    ds_rgb = xr.open_zarr(href, group="/measurements/reflectance/r20m")[["b02", "b03", "b04"]]
    ds_rgb = ds_rgb.sel(x=x_slice, y=y_slice)
    
    # Load SCL for cloud masking (already 20m)
    ds_scl = xr.open_zarr(href, group="/conditions/mask/l2a_classification/r20m")[["scl"]]
    ds_scl = ds_scl.sel(x=x_slice, y=y_slice)
    
    return xr.merge([ds_rgb, ds_scl])


def create_rgb_composite(hrefs, x_slice, y_slice):
    """
    Create median RGB composite from all scenes.
    """
    print(f"Loading RGB data from {len(hrefs)} scene(s)...")
    
    rgb_scenes = []
    for i, href in enumerate(hrefs):
        try:
            scene = load_rgb_scene(href, x_slice, y_slice)
            
            # Apply cloud mask
            valid = ~scene["scl"].isin([3, 8, 9])
            scene["b02"] = scene["b02"].where(valid)
            scene["b03"] = scene["b03"].where(valid)
            scene["b04"] = scene["b04"].where(valid)
            
            rgb_scenes.append(scene.compute())
            print(f"  [{i+1}/{len(hrefs)}] Loaded: {os.path.basename(href)}")
        except Exception as e:
            print(f"  [{i+1}/{len(hrefs)}] Error: {e}")
            continue
    
    if not rgb_scenes:
        return None
    
    if len(rgb_scenes) == 1:
        # Single scene
        rgb_median = xr.concat([
            rgb_scenes[0]["b04"],
            rgb_scenes[0]["b03"],
            rgb_scenes[0]["b02"],
        ], dim="band").assign_coords(band=["R", "G", "B"])
    else:
        # Stack and compute median
        r_stack = xr.concat([s["b04"] for s in rgb_scenes], dim="scene")
        g_stack = xr.concat([s["b03"] for s in rgb_scenes], dim="scene")
        b_stack = xr.concat([s["b02"] for s in rgb_scenes], dim="scene")
        
        rgb_median = xr.concat([
            r_stack.median(dim="scene", skipna=True),
            g_stack.median(dim="scene", skipna=True),
            b_stack.median(dim="scene", skipna=True),
        ], dim="band").assign_coords(band=["R", "G", "B"])
    
    print(f"✓ RGB composite created from {len(rgb_scenes)} scene(s)")
    return {"rgb_median": rgb_median, "scene_count": len(rgb_scenes)}


# Create RGB composite (independent from NDSI)
if hrefs:
    rgb_composite = create_rgb_composite(hrefs, x_slice, y_slice)
else:
    rgb_composite = None
    print("No scenes available for RGB processing.")

In [ ]:
# ======================================================================
# Visualization: Median RGB vs Snow/Ice Mask
# ======================================================================
# Uses independently loaded RGB data

if rgb_composite and composite:
    # Prepare RGB composite
    rgb_da = rgb_composite['rgb_median']
    rgb_np = np.stack([
        rgb_da.sel(band="R").values,
        rgb_da.sel(band="G").values,
        rgb_da.sel(band="B").values
    ], axis=-1)
    
    # Stretch RGB for visualization (2-98 percentile stretch)
    valid_vals = rgb_np[np.isfinite(rgb_np)]
    if valid_vals.size > 0:
        low, high = np.nanpercentile(valid_vals, [2, 98])
        rgb_stretched = (rgb_np - low) / (high - low + 1e-6)
        rgb_stretched = np.clip(rgb_stretched, 0, 1)
        rgb_stretched = np.nan_to_num(rgb_stretched, nan=0.0)
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # ----- Plot 1: RGB Composite -----
        axes[0].imshow(rgb_stretched)
        axes[0].set_title(f"Median RGB Composite\n(from {rgb_composite['scene_count']} scene(s))", 
                         fontsize=12, fontweight='bold')
        axes[0].axis('off')
        
        # ----- Plot 2: Snow/Ice Mask -----
        snow_data = composite['snow_mask'].values.astype(float)
        axes[1].imshow(snow_data, cmap='Blues', vmin=0, vmax=1)
        axes[1].set_title(f"Snow/Ice Mask\n(NDSI >= {NDSI_THRESHOLD})", 
                         fontsize=12, fontweight='bold')
        axes[1].axis('off')
        
        # Add percentage annotation
        axes[1].text(0.5, -0.05, f"{composite['snow_percentage']:.1f}% Snow Coverage",
                     transform=axes[1].transAxes, ha='center', fontsize=11,
                     bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
        
        plt.tight_layout()
        plt.show()
    else:
        print("RGB composite is empty after masking.")
else:
    print("RGB or NDSI composite missing. Run processing cells first.")